# HTTP
In deze opdracht ga je zelf HTTP requests doen.
Je gaat de webpagina opvragen die het cijfer bevat van het aantal coronabesmettingen van gisteren.

## Stap 1: de webpagina
We gaan een HTTP request doen van de volgende webpagina: 
**https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen**

Neem ook eens een kijkje door de volgende regel op te zoeken in je browser:
```
view-source:https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen
```

Kun je het cijfer vinden van het aantal besmettingen van gisteren?

## Stap 2: HTTP GET request
We gaan een HTTP GET request doen. Hiervoor gebruiken we de **urllib** library:

In [82]:
from urllib.request import urlopen

De URL (Uniform Resource Locator) van het HTTP GET request is:

In [83]:
url = 'https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen'

We gaan nu in Python een HTTP GET request doen:

In [84]:
r = urlopen(url)
the_html = r.read().decode('utf-8')

## Stap 3: response bekijken

Als eerste gaan we de status code bekijken

In [85]:
status_code = r.getcode()
print(status_code)

200


Is de status code 200? Dan is het gelukt!

Leg uit wat de volgende status codes betekenen:
- 404
- 414
- 418
- 429

Gelukkig bevat een response nog veel meer informatie:

In [86]:
for header, waarde in r.headers.items():
    print(header, waarde)

Content-Type text/html
Cache-Control no-cache, public
Expires Mon, 15 Mar 2021 16:37:40 GMT
Last-Modified Mon, 15 Mar 2021 16:09:33 GMT
ETag W/"604f86bd-1471f8"
Content-Security-Policy default-src 'self' statistiek.rijksoverheid.nl; img-src 'self' statistiek.rijksoverheid.nl data:; style-src 'self' 'unsafe-inline'; script-src 'self' statistiek.rijksoverheid.nl;
Referrer-Policy origin
X-Content-Type-Options nosniff
X-Frame-Options SAMEORIGIN
X-XSS-Protection 1; mode=block
Vary Accept-Encoding
Date Tue, 16 Mar 2021 13:44:18 GMT
Transfer-Encoding chunked
Connection close
Connection Transfer-Encoding
Strict-Transport-Security max-age=15768000 ; preload


Leg uit wat de volgende headers betekenen:
- Content-Type
- Last-Modified
- Date

Als laatste stap gaan we de HTML zelf bekijken. Hieronder tonen we de eerste 100 karakters van de HTML.

In [87]:
print(the_html[:100])

<!DOCTYPE html><html lang="nl" class="has-no-js"><head><script src="/init.js"></script><script nomod


## Stap 4: op zoek naar het aantal besmettingen

We gaan eerst de HTML inlezen met Python via de **xml** library.

In [88]:
import xml.etree.ElementTree as ET
from io import StringIO

In [89]:
the_html_as_file_object = StringIO(the_html)
tree = ET.parse(the_html_as_file_object)

In [90]:
html_element = 'div'
attribuut_waarde = 'infected'
html_elementen = tree.findall(f'.//{html_element}[@data-cy="{attribuut_waarde}"]')

Als het goed is, is er maar een HTML element *div* met als waarde van **data-cy**: **infected**

In [91]:
len(html_elementen)

1

In [92]:
element = html_elementen[0]
ET.dump(element)

<div class="kpi-value__StyledValue-sc-2mskn4-0 hZojKL" color="data.primary" data-cy="infected">5.549</div>


In [93]:
aantal_besmettingen = element.text.replace('.', '')
print(aantal_besmettingen)

5549


## Stap 5: JSON
HTML is niet echt makkelijk in te lezen door computers. Het RIVM biedt ook ander soorten formaten aan:
https://data.rivm.nl/covid-19/

We gaan nu een HTTP request doen om een JSON bestand op te vragen voor het aantal besmettingen per gemeente per dag.

In [94]:
import json

https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json

In [95]:
page = 'https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json'

In [96]:
r = urlopen(page)
json_as_string = r.read().decode('utf-8')
json_object = json.loads(json_as_string)

In [97]:
print(r.getcode())

200


In [98]:
stad = 'Amersfoort'
datum = '2020-04-10' # JAAR-MAAND-DAG
for informatie in json_object:
    if informatie['Municipality_name'] == stad:
        if informatie['Date_of_publication'] == datum:
            print(informatie)

{'Date_of_report': '2021-03-15 10:00:00', 'Date_of_publication': '2020-04-10', 'Municipality_code': 'GM0307', 'Municipality_name': 'Amersfoort', 'Province': 'Utrecht', 'Security_region_code': 'VR09', 'Security_region_name': 'Utrecht', 'Municipal_health_service': 'GGD Regio Utrecht', 'ROAZ_region': 'Traumazorgnetwerk Midden-Nederland', 'Total_reported': 2, 'Hospital_admission': 3, 'Deceased': 1}


* Hoeveel overledenen waren er in Amersfort op 27 november 2020? En hoeveel ziekenhuisopnames?
* Hoeveel positieve besmettingen had Urk op oudjaarsdag 2020?